In [1]:
import gzip
import pickle

In [2]:
with gzip.open("cache/engineers/harmonic_site_symmetries.pkl.gz", "rb") as f:
    data = pickle.load(f)

In [3]:
from preprocess_wychoffs import clasterize_harmonics

In [5]:
distances = clasterize_harmonics(data)

In [86]:
def assign_to_clusters(
    distances: pd.DataFrame):
    print(distances.index)
    assert distances.index == np.arange(distances.shape[0])
    assert distances.columns == np.arange(distances.shape[1])
    mapping = np.empty(distances.shape[0], dtype=int)
    remaining_distances = distances.copy()
    while not remaining_distances.empty:
        row, col = np.unravel_index(np.argmin(remaining_distances.values), remaining_distances.shape)
        row_label = remaining_distances.index[row]
        col_label = remaining_distances.columns[col]
        # enum -> cluster
        mapping[row_label] = col_label
        remaining_distances = remaining_distances.drop(row_label, axis=0).drop(col_label, axis=1)
    return mapping

In [87]:
distances.groupby(level=["spacegroup_number", "site_symmetries"]).apply(assign_to_clusters)

MultiIndex([(1, '1', 0)],
           names=['spacegroup_number', 'site_symmetries', 'sites_enumeration'])


AssertionError: 

In [79]:
distances

0         1  \
spacegroup_number site_symmetries sites_enumeration                       
1                 1               0                  0.714671  0.827096   
2                 -1              0                  0.121398  0.475568   
                                  1                  0.431300  0.555618   
                                  2                  0.391271  0.632527   
                                  3                  0.390799  0.646002   
...                                                       ...       ...   
230               -4..            0                  0.398577  0.538142   
                  .3.             0                  0.360647  0.501940   
                  2..             0                  0.319068  0.469515   
                  ..2             0                  0.405581  0.528810   
                  1               0                  0.360647  0.501940   

                                                            2         3  \
spacegroup_number site_symmetries sites_enumeration                       
1                 1               0                  0.726815  0.777794   
2                 -1              0                  0.366446  0.342288   
                                  1                  0.762396  0.636022   
                                  2                  0.364026  0.437183   
                                  3                  0.369742  0.451058   
...                                                       ...       ...   
230               -4..            0                  0.404988  0.510081   
                  .3.             0                  0.421940  0.495424   
                  2..             0                  0.403046  0.477250   
                  ..2             0                  0.448262  0.528838   
                  1               0                  0.421940  0.495424   

                                                            4         5  \
spacegroup_number site_symmetries sites_enumeration                       
1                 1               0                  0.652552  0.568427   
2                 -1              0                  0.404467  0.508501   
                                  1                  0.387974  0.684266   
                                  2                  0.625650  0.614571   
                                  3                  0.629636  0.614349   
...                                                       ...       ...   
230               -4..            0                  0.360983  0.076761   
                  .3.             0                  0.331377  0.171836   
                  2..             0                  0.287963  0.136417   
                  ..2             0                  0.342986  0.116684   
                  1               0                  0.331377  0.171836   

                                                            6         7  
spacegroup_number site_symmetries sites_enumeration                      
1                 1               0                  0.904852  0.748510  
2                 -1              0                  0.508906  0.677283  
                                  1                  0.285946  0.606475  
                                  2                  0.755734  0.847150  
                                  3                  0.756095  0.859236  
...                                                       ...       ...  
230               -4..            0                  0.707941  0.629005  
                  .3.             0                  0.660489  0.641264  
                  2..             0                  0.621178  0.598894  
                  ..2             0                  0.691767  0.636792  
                  1               0                  0.660489  0.641264  

[1731 rows x 8 columns]

In [65]:
mapping.shape

(8,)

In [64]:
mapping = np.empty(len(d), dtype=int)